In [8]:
import socket
import os

In [10]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

serverAddress = ('localhost', 2121)
sock.bind(serverAddress)
sock.listen(1)

while True:
    print('Waiting for a connection...')
    connection, clientAddress = sock.accept()
    print('Connection from', clientAddress)

    # For receiving data
    while True:
        data = connection.recv(1024)
        print('\nReceived', data.decode())

        if data:
            if data.decode()[:6] == 'DELETE':
                path = data[8:len(data)-1].strip()
                # Find the path file and delete the file
                try:
                    os.remove(path)
                    print('File deleted:', path)
                except OSError as e:
                    print(f"Error deleting file {path}: {e}")
            
            elif data.decode()[:8] == 'DOWNLOAD':
                # Download using only the path
#                 sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#                 serverAddress2 = ('localhost', 2222)
#                 sock2.bind(serverAddress2)
#                 sock2.listen(5)
#                 print('Listening...')
#                 newConn, newAddr = sock2.accept()
#                 print('Got connection from', newAddr)
#                 file_path = data.decode()[9:]
#                 if os.path.exists(file_path) and file_path.find("serverFolder"):
#                     print('Sending')
#                     with open(file_path, 'rb') as file:
#                         while True:
#                             file_data = file.read(1024)
#                             if not file_data: break
#                             newConn.sendall(file_data)
#                 else:
#                     print(file_path + ' Not Found.')
#                     newConn.sendall(b"File not found")
#                 newConn.close()
#                 sock2.close()

                # Download using only the name
                server_directory = r"C:\Users\parin\Desktop\NetworkFinalProject\serverFolder"
                sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                serverAddress2 = ('localhost', 2222)
                sock2.bind(serverAddress2)
                sock2.listen(5)
                print('Listening...')
                newConn, newAddr = sock2.accept()
                print('Got connection from', newAddr)
                file_name = data.decode()[9:]
                file_path = os.path.join(server_directory, file_name)
                if os.path.exists(file_path):
                    print('Sending')
                    with open(file_path, 'rb') as file:
                        while True:
                            file_data = file.read(1024)
                            if not file_data: break
                            newConn.sendall(file_data)
                else:
                    print(file_path + ' Not Found.')
                    newConn.sendall(b"File not found")
                newConn.close()
                sock2.close()
                    
            elif data.decode()[:4] == 'LIST':
                try:
                    # os.getcwd()
                    directory_path = r"C:\Users\parin\Desktop\NetworkFinalProject\serverFolder" 
                    file_list = [f"{filename} - {os.path.getsize(os.path.join(directory_path, filename))} bytes" for filename in os.listdir(directory_path)]
                    print('Sending file list:', file_list)
                    connection.sendall('\n'.join(file_list).encode())
                except FileNotFoundError:
                    print(f"Directory not found: {directory_path}")
                    connection.sendall("Directory not found".encode())
                    
            elif data.decode()[:3] == 'PWD':
                try:
                    directory_path = os.getcwd()
                    connection.sendall(directory_path.encode())
                except FileNotFoundError:
                    print(f"Directory not found: {directory_path}")
                    connection.sendall("Directory not found".encode())

            elif data.decode()[:6] == 'UPLOAD':
                file_name = data.decode()[7:]
                sock2 = socket.socket()
                sock2.bind(('localhost', 2222))
                sock2.listen(5)
                print('Listening...')
                newConn, newAddr = sock2.accept()
                print('Got connection from', newAddr)
                server_folder = r"C:\Users\parin\Desktop\NetworkFinalProject\serverFolder"
                file_path = os.path.join(server_folder, file_name)
                with open(file_path, 'wb') as file:
                    while True:
                        file_data = newConn.recv(1024)
                        if not file_data: break
                        file.write(file_data)
                    print('---DONE---')
                newConn.close()
                sock2.close()
                
            elif data == 'SHUTDOWN'.encode(): # When the client enters 'shutdown', the server shuts down
                break
        else:
            print('No more data from', clientAddress)
            break
    connection.close()
    if data == 'SHUTDOWN'.encode(): break
        
print('Shutting down the server.')        
sock.close()

Waiting for a connection...
Connection from ('127.0.0.1', 13830)

Received SHUTDOWN
Shutting down the server.
